# Text Pre-Processor

This is a component that process text for natural language processing using multiple packages.

This notebook shows:
    how to use the functionalities from the text processor 

## Imports

In [8]:
import nltk
import spacy
import os
import re
import langdetect
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


## NLP Master Class  Definition

In [9]:
class NLP():
    """
    This class is the moder of all nlp classes.

    """
    # Class Attribute

    # Initializer / Instance Attributes
    def __init__(self, texts_dir):
        self.texts_dir = texts_dir
        self.text_list = self.return_texts_dir()

    def return_texts_dir(self):
        """
        :return: Get a list of all the texts on the dir
        """
        all_files = os.listdir(self.texts_dir)
        text_list = []
        for file in all_files:
            text_list = text_list + [open(self.texts_dir + file, "r", encoding="utf-8").read()]
        return text_list

    def get_unique_name_tokens(self):
        """
        :return: All the tokens in the list of text without repetition
        """
        vectorizer =  CountVectorizer()
        vectorized_object = vectorizer.fit_transform(self.text_list)
        return vectorizer.get_feature_names()

    def get_main_language(self):
        """
        :return: The main language of the text
        """
        all_text = " ".join(self.text_list)
        return langdetect.detect(all_text)

    def get_present_languages(self):
        """
        :return: The languages detected on the text by probability score
        """
        all_text = " ".join(self.text_list)
        return langdetect.detect_langs(all_text)

    def get_all_words_with_end(self, word_end):
        """

        :param word_end: the string of end to search on the text list
        :return: the words in which the word end is present
        """
        word_end_list = []
        for word in self.get_unique_name_tokens():
            if word.endswith(word_end):
                word_end_list.append(word)
        return word_end_list

    def get_all_words_with_start(self, word_start):
        """
        :param word_start: the string of start to search on the text list
        :return: the words in which the word start is present
        """
        word_start_list = []
        for word in self.get_unique_name_tokens():
            if word.startswith(word_start):
                word_start_list.append(word)
        # word_end_list = [word_end_list.append(word) for word in vocabulario if word.endswith(sufixo)]
        return word_start_list

    def organize_ngrams(self, ngrams=2):
        """

        :param ngrams: number of words in the gram
        :return: the text list organized in n-grams

        """

        vocab = defaultdict(int)
        for sample in self.text_list:
            tokens = re.findall(r'[\w-]+', sample.lower())
            for start in range(len(tokens) - ngrams):
                end = start + ngrams
                tok = ' '.join(tokens[start:end])
                vocab[tok] += 1
                # if tok not in ngrams_tokens_list:
                #     ngrams_tokens_list.append(tok)
        return list(vocab.keys())

    def top_ngrams(self, ngrams=2, number_best_tokens=10):
        """
        :param ngrams: number of words in the gram
        :param number_best_tokens: The number of best tokens returned
        :return: the top ngrams from your text list

        """

        vocab = defaultdict(int)
        for sample in self.text_list:
            tokens = re.findall(r'[\w-]+', sample.lower())
            for start in range(len(tokens) - ngrams):
                end = start + ngrams
                tok = ' '.join(tokens[start:end])
                vocab[tok] += 1
        all_tokens = sorted(vocab.items(), key=lambda i: i[1], reverse=True)
        best_tokens = all_tokens[:number_best_tokens]
        return best_tokens


# NLP Text Pre Processment Definition

In [10]:
class PreProcessament(NLP):
    """
    This class is only for text pre-processment functionalities
    """
    # Initializer / Instance Attributes
    def __init__(self,dir_texts):
        NLP.__init__(self, dir_texts)
        self.main_language = self.get_main_language()
        self.doc_list_spacy = self.set_doc_list_spacy()
        self.token_names_matrix_spacy = self.get_token_names_matrix()

    def get_token_numbers_matrix(self,TF_IDF = False):
        """

        :param TF_IDF: If true matrix tf_idf se counting matrix
        :return: Tokens matrix from sklearn
        tf-idf: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
        counts:https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

        """
        if TF_IDF:
            vectorizer = TfidfVectorizer()
        else:
            vectorizer = CountVectorizer()
        vectorized_object = vectorizer.fit_transform(self.text_list)
        return vectorized_object.toarray()

    def set_doc_list_spacy(self):
        """
        :return: list of docs format spacy

        """
        language = 'pt_core_news_sm' if (self.main_language == 'pt') else 'en'
        nlp = spacy.load(language)
        docs_list = []
        for text in self.text_list:
            docs_list.append(nlp(text))

        return docs_list


    def get_token_names_matrix(self,punctuation=False,repeated_tokens=True,all_low_cap = True, token_spacy= True):
        
        """
        :param punctuation:If False punctuation (.,!? etc) removed else keeped
        :param repeated_tokens:If True keep repeatables for text else remove
        :param all_low_cap:If True return everything low cap
        :param token_spacy:IF True return token_spacy else string
        :return: Matrix of nominal tokens. If token_pacy = True is a type from Spacy else returns string
        """
        language = 'pt_core_news_sm' if (self.main_language == 'pt') else 'en'
        nlp = spacy.load(language)
        name_tokens_list = []
        for i, text in enumerate(self.text_list):
            text = text if repeated_tokens else list(set(text))
            text = text.lower() if all_low_cap else text
            doc = nlp(text)
            if token_spacy:
                name_tokens_list.append([token for token in doc] if punctuation else [token for token in doc if not token.is_punct])
            else:
                name_tokens_list.append([token.orth_  for token in doc] if punctuation else [token.orth_  for token in doc if not token.is_punct])
        return name_tokens_list

    def split_text_into_sentences(self):
        """
        :return: All the text joined
        """

        all_text = " ".join(self.text_list)
        return nltk.tokenize.sent_tokenize(all_text)


    def get_stop_words(self, language_stop_words):
        """
        :param language_stop_words: language in string format
        :return: Stop Words from  nltk package
        """

        stopwords = nltk.corpus.stopwords.words(language_stop_words)
        return stopwords

    def remove_stop_words_vocab(self, stop_words, vocab):
        """

        :param stop_words: words to remove
        :param vocab: The words present in all sentences
        :return: Vocab without stop words
        """

        return list(set(vocab) - set(stop_words))

    def tag_pos_list(self):
        """
        :return: Part of speech(POS) with accuracy of 79.94%
        """

        pos_list = []
        for doc in self.token_names_matrix_spacy:
            pos_list.append([(token.orth_, token.pos_) for token in doc])
        return pos_list

    def get_lemmatization(self,pos_list = ['NOUN','VERB','ADJ','NUM']):
        """
        :param pos_list: List of part of speech as classified in spa
        :return: List of lematized words according with the POS passed pos_list
        """

        lemmas_list = []
        for doc in self.token_names_matrix_spacy:
            lemmas_list.append([token.lemma_ for token in doc  if token.pos_ in pos_list])
        return lemmas_list

    def recognize_entyties(self):
        """
        :return: List of entities for each text
        """

        entyties_list = []
        for doc in self.doc_list_spacy:
            entyties_list.append([(entity, entity.label_) for entity in doc.ents])
        return entyties_list

    def check_text_similarity(self,word1, word2):
        """

        :param word1: first word
        :param word2:  second word
        :return: Similarity value from two words computed by spacy
        """

        language = 'pt_core_news_sm' if (self.main_language == 'pt') else 'en'
        nlp = spacy.load(language)
        doc = nlp(word1 + ' ' + word2)
        print(f'The similarity between {word1} and {word2} is: ', doc[0].similarity(doc[1]))



# Class Instantiation

Criação do objeto preprocessor

In [8]:
preprocessor = PreProcessament("textos_internet/noticias_agronegocio/")
preprocessor

# NLP Masterclass Funtionalities

## Return texts on the directory
 Get a list of all the texts on the dir

In [11]:
preprocessor.return_texts_dir()[:1]

['Confira a previsão do tempo para a primeira semana do outono\nSomar Meteorologia indica que região do país terá chuva pesada nos próximos dias, com pancadas fortes e trovoadas; veja onde!\nMarch 22, 2020 18:18  |  Redação - Canal Rural\nChuva forte atinge propriedade rural\n    \nSegunda-feira, 21\nSul\nAinda há a ação de uma região de alta pressão atmosférica no oceano, que joga ventos úmidos do mar contra a costa. Assim, há condição para instabilidades na faixa leste, com eventual garoa e nevoeiros entre o leste do Paraná, Santa Catarina e litoral norte gaúcho. \n \nNo interior, o destaque passa a ser o sol. Com maior amplitude térmica, a manhã fica um pouco mais fria, mas a tarde segue quente especialmente nas áreas de fronteira com a Argentina.\n \nSudeste\nÁreas de chuva mais pesada ficam concentradas no norte mineiro, com condição ainda para trovoadas e elevados volumes acumulados. Nos demais estados do Sudeste, a intensidade da chuva já diminui em relação aos dias anteriores e

## Unique names toquens
All the tokens in the list of text without repetition

In [16]:
preprocessor.get_unique_name_tokens()[:10]

['08h25more_horiz',
 '10',
 '100',
 '10h21',
 '10h41more_horiz',
 '11',
 '11h23more_horiz',
 '12',
 '123',
 '12h29more_horiz']

## Get main language on the text


In [17]:
preprocessor.get_main_language()

'pt'

## Remoção de Stop Words do vocabulário

Subtração da string dos stop words em relação ao vocabulário

In [18]:
preprocessor.get_present_languages()

[pt:0.9999955505359109]

## The words in which the word start is present

In [12]:
preprocessor.get_all_words_with_start('in')[:5]

['in', 'inc', 'incentivando', 'incerteza', 'incertezas']

## The words in which the word end is present

In [19]:
preprocessor.get_all_words_with_end('inho')

['brunadinho', 'vizinho']

## The text list organized in n-grams

In [24]:
preprocessor.organize_ngrams(ngrams=4)[:5]

['confira a previsão do',
 'a previsão do tempo',
 'previsão do tempo para',
 'do tempo para a',
 'tempo para a primeira']

## The top ngrams from your text list

In [25]:
preprocessor.top_ngrams(ngrams=4, number_best_tokens=10)

[('no momento do embarque', 3),
 ('prevê que a safra', 3),
 ('que a safra brasileira', 3),
 ('a maior da série', 3),
 ('maior da série histórica', 3),
 ('da série histórica do', 3),
 ('série histórica do instituto', 3),
 ('histórica do instituto iniciada', 3),
 ('do instituto iniciada em', 3),
 ('instituto iniciada em 1975', 3)]

# TextPre Processment Funtionalities

## Token Numbers Matrix with counting strategy

In [26]:
preprocessor.get_token_numbers_matrix(TF_IDF = False)

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Token Numbers Matrix with TF-IDF strategy

In [27]:
preprocessor.get_token_numbers_matrix(TF_IDF = True)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.04233072,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02436815, 0.        ,
        0.02866533],
       ...,
       [0.        , 0.04783516, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Set Doc List to Spacy

In [30]:
preprocessor.set_doc_list_spacy()[:1]

[Confira a previsão do tempo para a primeira semana do outono
 Somar Meteorologia indica que região do país terá chuva pesada nos próximos dias, com pancadas fortes e trovoadas; veja onde!
 March 22, 2020 18:18  |  Redação - Canal Rural
 Chuva forte atinge propriedade rural
     
 Segunda-feira, 21
 Sul
 Ainda há a ação de uma região de alta pressão atmosférica no oceano, que joga ventos úmidos do mar contra a costa. Assim, há condição para instabilidades na faixa leste, com eventual garoa e nevoeiros entre o leste do Paraná, Santa Catarina e litoral norte gaúcho. 
  
 No interior, o destaque passa a ser o sol. Com maior amplitude térmica, a manhã fica um pouco mais fria, mas a tarde segue quente especialmente nas áreas de fronteira com a Argentina.
  
 Sudeste
 Áreas de chuva mais pesada ficam concentradas no norte mineiro, com condição ainda para trovoadas e elevados volumes acumulados. Nos demais estados do Sudeste, a intensidade da chuva já diminui em relação aos dias anteriores e 

## Matrix of nominal tokens

In [32]:
preprocessor.get_token_names_matrix(punctuation=False,repeated_tokens=True,all_low_cap = True, token_spacy= True)[:1]

[[confira,
  a,
  previsão,
  do,
  tempo,
  para,
  a,
  primeira,
  semana,
  do,
  outono,
  ,
  somar,
  meteorologia,
  indica,
  que,
  região,
  do,
  país,
  terá,
  chuva,
  pesada,
  nos,
  próximos,
  dias,
  com,
  pancadas,
  fortes,
  e,
  trovoadas,
  veja,
  onde,
  ,
  march,
  22,
  2020,
  18:18,
   ,
  |,
   ,
  redação,
  canal,
  rural,
  ,
  chuva,
  forte,
  atinge,
  propriedade,
  rural,
  
      ,
  segunda-feira,
  21,
  ,
  sul,
  ,
  ainda,
  há,
  a,
  ação,
  de,
  uma,
  região,
  de,
  alta,
  pressão,
  atmosférica,
  no,
  oceano,
  que,
  joga,
  ventos,
  úmidos,
  do,
  mar,
  contra,
  a,
  costa,
  assim,
  há,
  condição,
  para,
  instabilidades,
  na,
  faixa,
  leste,
  com,
  eventual,
  garoa,
  e,
  nevoeiros,
  entre,
  o,
  leste,
  do,
  paraná,
  santa,
  catarina,
  e,
  litoral,
  norte,
  gaúcho,
  
   ,
  no,
  interior,
  o,
  destaque,
  passa,
  a,
  ser,
  o,
  sol,
  com,
  maior,
  amplitude,
  térmica,
  a,
  manhã,
  fica,

## All the text joined in one string

In [35]:
preprocessor.split_text_into_sentences()[:2]

['Confira a previsão do tempo para a primeira semana do outono\nSomar Meteorologia indica que região do país terá chuva pesada nos próximos dias, com pancadas fortes e trovoadas; veja onde!',
 'March 22, 2020 18:18  |  Redação - Canal Rural\nChuva forte atinge propriedade rural\n    \nSegunda-feira, 21\nSul\nAinda há a ação de uma região de alta pressão atmosférica no oceano, que joga ventos úmidos do mar contra a costa.']

## Removing stop words from vocab

In [36]:
stop_words = preprocessor.get_stop_words('portuguese')
vocab = preprocessor.get_unique_name_tokens()
preprocessor.remove_stop_words_vocab(stop_words, vocab)[:20]


['access_time11',
 'aparente',
 'mexer',
 'ressaltou',
 'produção',
 'último',
 'longa',
 '96',
 'mudar',
 'quedas',
 'rapidamente',
 'aumentam',
 'incluem',
 'sucessos',
 'exemplo',
 'confira',
 'imune',
 'causada',
 'cadeia',
 'segundo']

## Part of speech(POS) Tagging

In [39]:
preprocessor.tag_pos_list()[:1]

[[('confira', 'VERB'),
  ('a', 'DET'),
  ('previsão', 'NOUN'),
  ('do', 'ADP'),
  ('tempo', 'NOUN'),
  ('para', 'ADP'),
  ('a', 'DET'),
  ('primeira', 'ADJ'),
  ('semana', 'NOUN'),
  ('do', 'PROPN'),
  ('outono', 'PROPN'),
  ('\n', 'SPACE'),
  ('somar', 'PROPN'),
  ('meteorologia', 'NOUN'),
  ('indica', 'VERB'),
  ('que', 'SCONJ'),
  ('região', 'PROPN'),
  ('do', 'DET'),
  ('país', 'NOUN'),
  ('terá', 'VERB'),
  ('chuva', 'NOUN'),
  ('pesada', 'VERB'),
  ('nos', 'PRON'),
  ('próximos', 'ADJ'),
  ('dias', 'NOUN'),
  ('com', 'ADP'),
  ('pancadas', 'NOUN'),
  ('fortes', 'ADJ'),
  ('e', 'CCONJ'),
  ('trovoadas', 'ADJ'),
  ('veja', 'VERB'),
  ('onde', 'ADV'),
  ('\n', 'SPACE'),
  ('march', 'PROPN'),
  ('22', 'NUM'),
  ('2020', 'NUM'),
  ('18:18', 'NUM'),
  (' ', 'SPACE'),
  ('|', 'X'),
  (' ', 'SPACE'),
  ('redação', 'NOUN'),
  ('canal', 'PROPN'),
  ('rural', 'PROPN'),
  ('\n', 'SPACE'),
  ('chuva', 'PROPN'),
  ('forte', 'ADJ'),
  ('atinge', 'VERB'),
  ('propriedade', 'NOUN'),
  ('rural', '

## Lemmas 
List of lematized words according with the POS passed pos_list

In [40]:
preprocessor.get_lemmatization(pos_list = ['NOUN','VERB','ADJ','NUM'])[:1]

[['conferir',
  'previsão',
  'tempo',
  'primeiro',
  'semana',
  'meteorologia',
  'indicar',
  'país',
  'ter',
  'chuva',
  'pesar',
  'próximo',
  'dia',
  'pancada',
  'forte',
  'trovoar',
  'ver',
  '22',
  '2020',
  '18:18',
  'redação',
  'forte',
  'atingir',
  'propriedade',
  'rural',
  '21',
  'sul',
  'haver',
  'ação',
  'região',
  'alto',
  'pressão',
  'atmosférico',
  'jogar',
  'vento',
  'úmidos',
  'do',
  'mar',
  'costa',
  'haver',
  'condição',
  'instabilidade',
  'ler',
  'eventual',
  'garoa',
  'nevoeiro',
  'ler',
  'interior',
  'destacar',
  'ser',
  'sol',
  'maior',
  'amplitude',
  'térmico',
  'manhã',
  'ficar',
  'frio',
  'tardar',
  'seguir',
  'área',
  'fronteiro',
  'sudeste',
  'área',
  'chuva',
  'pesar',
  'ficar',
  'concentrar',
  'norte',
  'mineiro',
  'condição',
  'trovoar',
  'elevar',
  'volume',
  'acumular',
  'estar',
  'sudeste',
  'intensidade',
  'chuva',
  'diminuir',
  'relação',
  'anterior',
  'chuva',
  'ocorrer',
  'f

## Entyties recognition

In [41]:
preprocessor.recognize_entyties()[:1]

[[(Somar Meteorologia, 'PER'),
  (March 22, 'MISC'),
  (Redação - Canal Rural, 'MISC'),
  (Chuva, 'PER'),
  (Sul, 'LOC'),
  (Paraná, 'LOC'),
  (Santa Catarina, 'LOC'),
  (Argentina, 'LOC'),
  (Sudeste, 'LOC'),
  (Áreas, 'LOC'),
  (Sudeste, 'LOC'),
  (Vale do Paraíba, 'LOC'),
  (São Paulo, 'LOC'),
  (Rio de Janeiro, 'LOC'),
  (Centro-Oeste, 'LOC'),
  (Mato Grosso, 'LOC'),
  (Goiás, 'LOC'),
  (Distrito Federal, 'LOC'),
  (Mato Grosso, 'LOC'),
  (“, 'LOC'),
  (Distrito Federal, 'LOC'),
  (Mato Grosso do Sul”, 'LOC'),
  (Somar, 'LOC'),
  (Nordeste, 'LOC'),
  (Bahia, 'LOC'),
  (Nordeste, 'LOC'),
  (Norte, 'LOC'),
  (Norte, 'LOC'),
  (Pará, 'LOC'),
  (Tocantins, 'LOC'),
  (Amazônia, 'LOC'),
  (Amazonas, 'LOC'),
  (Pará, 'LOC')]]

In [ ]:
## Check if there is some similarity between two words
Similarity goes from 0 to 1

In [45]:
preprocessor.check_text_similarity('falar','falar')

1.0
